In [ ]:
import os
import torch
from transformers import PegasusConfig, PegasusForConditionalGeneration, PegasusTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

def create_small_pegasus_config():
    '''Create a smaller Pegasus configuration suitable for training on 3000 samples.'''
    config = PegasusConfig(
        vocab_size=96103,  # Original vocab size for tokenizer compatibility
        encoder_layers=6,  # Reduced from 16
        decoder_layers=6,  # Reduced from 16
        encoder_attention_heads=8,  # Reduced from 16
        decoder_attention_heads=8,  # Reduced from 16
        encoder_ffn_dim=2048,  # Reduced from 4096
        decoder_ffn_dim=2048,  # Reduced from 4096
        d_model=512,  # Reduced from 1024
        max_position_embeddings=512,  # Reduced context length
        pad_token_id=0,
        eos_token_id=1,
        forced_eos_token_id=1,
        activation_function='gelu',
        dropout=0.2,  # Increased dropout for smaller dataset
        attention_dropout=0.2,
        activation_dropout=0.2,
        num_beams=4,
        encoder_layerdrop=0.1,  # Added layerdrop for regularization
        decoder_layerdrop=0.1,
        scale_embedding=True,
        use_cache=True,
        is_encoder_decoder=True
    )
    return config

TRAINING_PARAMS = {
    'MAX_INPUT_LENGTH': 512,
    'MAX_TARGET_LENGTH': 64,
    'BATCH_SIZE': 8,
    'GRADIENT_ACCUMULATION_STEPS': 2,
    'NUM_EPOCHS': 15,
    'LEARNING_RATE': 1e-4,
    'WARMUP_RATIO': 0.1,
    'WEIGHT_DECAY': 0.01,
    'EARLY_STOPPING_PATIENCE': 3,
    'EVAL_STEPS': 50,
}

def set_seed(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_data(num_samples=3000):
    dataset = load_dataset('cnn_dailymail', '3.0.0')
    full_train_data = dataset['train'].select(range(num_samples))
    train_size = int(0.9 * len(full_train_data))
    train_data = full_train_data.select(range(train_size))
    val_data = full_train_data.select(range(train_size, len(full_train_data)))
    return train_data, val_data

class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data[idx]['article']
        summary = self.data[idx]['highlights']
        inputs = self.tokenizer(article, max_length=self.max_input_length, truncation=True, padding='max_length', return_tensors='pt')
        targets = self.tokenizer(summary, max_length=self.max_target_length, truncation=True, padding='max_length', return_tensors='pt')
        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': targets.input_ids.squeeze()
        }

def create_dataloaders(train_data, val_data, tokenizer, batch_size):
    train_dataset = SummarizationDataset(train_data, tokenizer, TRAINING_PARAMS['MAX_INPUT_LENGTH'], TRAINING_PARAMS['MAX_TARGET_LENGTH'])
    val_dataset = SummarizationDataset(val_data, tokenizer, TRAINING_PARAMS['MAX_INPUT_LENGTH'], TRAINING_PARAMS['MAX_TARGET_LENGTH'])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

def evaluate(model, data_loader, tokenizer, device):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=TRAINING_PARAMS['MAX_TARGET_LENGTH'])

            for i in range(len(input_ids)):
                reference = tokenizer.decode(labels[i], skip_special_tokens=True)
                generated_summary = tokenizer.decode(generated_ids[i], skip_special_tokens=True)

                rouge_result = scorer.score(reference, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(rouge_result[metric].fmeasure)

    avg_rouge = {metric: sum(scores) / len(scores) for metric, scores in rouge_scores.items()}

    return avg_rouge



def train_model(model, train_loader, val_loader, tokenizer, device, num_epochs):
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': TRAINING_PARAMS['WEIGHT_DECAY'],
        },
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=TRAINING_PARAMS['LEARNING_RATE'])
    num_training_steps = len(train_loader) * num_epochs
    num_warmup_steps = int(num_training_steps * TRAINING_PARAMS['WARMUP_RATIO'])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

    best_val_loss = float('inf')
    best_model_path = '/Users/pratheeshjp/Desktop/NLP-Project/best_model.pth'
    patience = TRAINING_PARAMS['EARLY_STOPPING_PATIENCE']
    no_improve = 0

    train_losses = []
    val_losses = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for i, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS']  
            loss.backward()

            if (i + 1) % TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS'] == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad() 

            total_train_loss += loss.item() * TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS']  

            
            progress_bar.set_postfix({'loss': total_train_loss / (i+1)})

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)



        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)



        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Average Train Loss: {avg_train_loss:.4f}")
        print(f"Average Val Loss: {avg_val_loss:.4f}")
        print(f"Learning Rate: {scheduler.get_last_lr()[0]:.2e}")


                # Evaluate every 2 epochs
        if (epoch + 1) % 2 == 0:
            print(f"Evaluating after epoch {epoch+1}...")
            current_rouge = evaluate(model, val_loader, tokenizer, device)
            for metric in rouge_scores:
                rouge_scores[metric].append(current_rouge[metric])
            print(f"Current ROUGE Scores: {current_rouge}")


        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

    end_time = time.time()
    training_time = end_time - start_time

    return train_losses, val_losses, rouge_scores, training_time


def plot_training_progress(train_losses, val_losses, rouge_scores):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 3)
    for metric, scores in rouge_scores.items():
        plt.plot(scores, label=metric)
    plt.legend()
    plt.title('ROUGE Scores')

    plt.tight_layout()
    plt.savefig('training_progress.png')
    plt.close()
    plt.show()


def generate_summary(model, article, tokenizer, device, max_length=128):
    inputs = tokenizer(article, max_length=1024, truncation=True, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def main():
    set_seed()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
    config = create_small_pegasus_config()
    model = PegasusForConditionalGeneration(config).to(device)
    print("Total number of parameter in the model are :")
    print(sum(p.numel() for p in model.parameters()))
    
    train_data, val_data = load_data(num_samples=3000)
    train_loader, val_loader = create_dataloaders(train_data, val_data, tokenizer, TRAINING_PARAMS['BATCH_SIZE'])
    train_losses, val_losses, rouge_scores, training_time = train_model(model, train_loader, val_loader, tokenizer, device, TRAINING_PARAMS['NUM_EPOCHS'])
    plot_training_progress(train_losses, val_losses,rouge_scores)

    print("Evaluating fine-tuned model...")
    fine_tuned_bleu, fine_tuned_rouge = evaluate(model, val_loader, tokenizer, device)
    print("Fine-tuned Model Performance:")
    print(f"BLEU Score: {fine_tuned_bleu}")
    print(f"ROUGE Scores: {fine_tuned_rouge}")
    print(f"Total training time: {training_time:.2f} seconds")

    print("\nGenerating example summaries...")
    for i in range(3):
        article = val_data[i]["article"]
        reference = val_data[i]["highlights"]
        generated = generate_summary(model, article, tokenizer, device)
        print(f"\nArticle {i+1}:")
        print(f"Reference: {reference}")
        print(f"Generated: {generated}")
        print("-" * 50)

if __name__ == '__main__':
    main()


KeyboardInterrupt: 

In [12]:
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
    config = create_small_pegasus_config()
    model = PegasusForConditionalGeneration(config).to("mps")
    print("Total number of parameter in the model are :")
    print(sum(p.numel() for p in model.parameters()))

Total number of parameter in the model are :
93869568


/Users/pratheeshjp/Desktop/NLP-Project


Shared/      pratheeshjp/
